pip install git+https://github.com/huggingface/transformers.git

# Obtaining a Dataset
For this tutorial, they've decided the data should be in the format ChatML (Explore more into what ChatML is later)
They illustrate multiple ways of obtaining a dataset
1. download a preprocessed data set, OpenAssistant dataset is already in the ChatML format and does not need any preprocessing on our end
2. Download a dataset thats not in our format. This will need to be inspected and then changed to our format. See how in article
3. Take a transcript and format into ChatML format. See how in article
4. Take raw text and convert to ChatML format

This list goes from least work to most work
I will just use the easiest option.

In [22]:
import os
from datasets import load_dataset
#Open Assistant Dataset
dataset = load_dataset("OpenAssistant/oasst_top1_2023-08-25")

# Load and Prepare model and tokenizer
load model and tokenizer and make sure they "understand" the ChatML format

## ChatML tags
<|im_start|> <--- start of prompt token\
<|im_end|> <--- end of prompt token

In [27]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model_id = "google/gemma-2b-it"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)  

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

# Prepare data for training
tokenizing the data

In [ ]:
def tokenize(element):
    return tokenizer(
        element["text"],
        truncation=True,
        max_length=2048, #sets max length of a data sample (this is not words but tokens)
        add_special_tokens=False,
    )

dataset_tokenized = dataset.map(
    tokenize, 
    batched=True, 
    num_proc=os.cpu_count(),    # multithreaded
    remove_columns=["text"]     # don't need the strings anymore, we have tokens from here on
)